In [37]:
from langchain_core.tools import BaseTool
from typing import Optional
from langchain_core.callbacks import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun

class MyCustomTool(BaseTool):
    name: str = "my_custom_tool"
    description: str = "A custom tool that performs a specific task."

    def _run(
        self,
        input_text: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        # Implement your tool's functionality here
        return f"Processed input: {input_text}"

    async def _arun(
        self,
        input_text: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        # Asynchronous implementation (optional)
        return self._run(input_text)

In [68]:
from fastmcp import Client
from litellm.experimental_mcp_client.tools import transform_mcp_tool_to_openai_tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.messages import ToolMessage
from langchain_core.runnables import Runnable
from langchain_core.agents import AgentAction, AgentFinish
from langchain.agents.output_parsers.tools import ToolAgentAction
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain_core.agents import AgentAction
from dotenv import load_dotenv

load_dotenv(override=True)

async def main(input_text='Summarize my notes about complexity'):
    mcp_client = Client("http://localhost:8000/mcp/")
    
    async with mcp_client:
        # Fetch and transform tools
        tools = await mcp_client.list_tools()
        openai_tools = [transform_mcp_tool_to_openai_tool(t) for t in tools]

        # Initialize LLM with tools
        llm = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(openai_tools)

        # Define the prompt template
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant that uses tools to answer user questions."),
            ("user", "{input}"),
            MessagesPlaceholder("agent_scratchpad"),
        ])

        # Define the agent using the prompt, LLM, and output parser
        agent_runnable: Runnable = prompt | llm | OpenAIToolsAgentOutputParser()

        scratchpad_steps: list[tuple[AgentAction, str]] = []

        # First reasoning step
        step = await agent_runnable.ainvoke({
            "input": input_text,
            "agent_scratchpad": format_to_openai_tool_messages(scratchpad_steps),
        })

        # Loop until final output
        while isinstance(step[0], AgentAction) or isinstance(step[0], ToolAgentAction):
            print(f"\n🔧 Tool call: {step[0].tool}({step[0].tool_input})")

            # Call the tool via MCP
            tool_output = await mcp_client.call_tool(step[0].tool, step[0].tool_input)

            print(f"✅ Tool result: {tool_output[:300]}...")

            # Record the (action, result) pair
            scratchpad_steps.append((step[0], tool_output))

            # Continue reasoning
            step = await agent_runnable.ainvoke({
                "input": input_text,
                "agent_scratchpad": format_to_openai_tool_messages(scratchpad_steps),
            })
            # Final message (AgentFinish)
            print("\n🎯 FINAL RESPONSE:")
            print(step)
            return step


response=await main()


🔧 Tool call: get_notes_for_summary({'query': 'complexity', 'max_notes': 5})
✅ Tool result: [TextContent(type='text', text='Notes about \'complexity\' (showing 3449 results):\n\nNote ID: c139a9396e82239954f6b259cefb979c\nTitle: Disorganized complexity\nFolder: complex_systems\nSimilarity: 0.5355\n\nIn Warren Weaver\'s view, disorganized complexity results from the particular system having a very large number of parts, say millions of parts, or many more. Though the interactions of the parts in a "disorganized complexity" situation can be seen as largely random, the properties of the system as a whole can be understood by using probability and statistical methods. Individual fluctuations cancel out and system can be predictably described.\n\n"Complex Adaptive Systems: An Introduction to Computational Models of Social Life", John H. Miller,\xa0Scott E. Page, page 47\n\n[[Complex system is greater than sum of its parts]]\n\n#complex_system \n#complexity\n#society \n---\nNote ID: 7c997db5d

In [69]:
print(response.return_values['output'])

Here are the summaries of your notes about 'complexity':

1. **Disorganized complexity**: This concept, as per Warren Weaver's view, results from a system having a very large number of parts, say millions or more. The interactions of the parts in a "disorganized complexity" situation can be seen as largely random, but the properties of the system as a whole can be understood by using probability and statistical methods. Individual fluctuations cancel out and the system can be predictably described.

2. **Complexity is the outcome of uncertainty minimization**: Research by Krakauer, Flack, and others has shown that the complexity and multiscale structures of biological systems are predictable outcomes of evolutionary dynamics driven by uncertainty minimization.

3. **Hierarchies are needed for efficient information processing**: Complex systems can evolve from simple systems only if there are stable intermediate forms. The resulting complex forms will naturally be hierarchic. Hierarchie

In [70]:
response=await main('Summarize my notes about AI')
print(response.return_values['output'])


🔧 Tool call: get_notes_for_summary({'query': 'AI', 'max_notes': 5})
✅ Tool result: [TextContent(type='text', text='Notes about \'AI\' (showing 3957 results):\n\nNote ID: 9501bb55c2c54e944c804ad8ca179a4b\nTitle: Humans surrender autonomy when we use autonomous machines\nFolder: AI\nSimilarity: 0.4437\n\n "... the computer scientist Joseph Weizenbaum, the inventor of the very first AI mirror, the ELIZA chatbot of the mid-1960s. A decade later, Weizenbaum noted the paradox of humans surrendering their belief in their own autonomy at the very moment that they chose to rely on autonomous machines that they themselves made."\nSource: Weizenbaum, Joseph.\xa0_Computer Power and Human Reason_. and "The AI Mirror: How to Reclaim Our Humanity in an Age of Machine Thinking", Shannon Vallor, page 170\n\n[[We shape our tools and thereafter they shape us]]\n[[We should avoid tyranny narrow visions]]\n\n#AI \n#AI_hype \n#ai_hiring \n#autonomy\n\n---\nNote ID: a0f7b458c2263ec75ac73f3e01c6ff4b\nTitle: 